In [1]:
%pip install -q numpy pandas datasets transformers tqdm safetensors scikit-learn

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [22]:
DATASET_DIR_PATH = "/gdrive/MyDrive/ФТиАД/ВКР/dataset"
FILE_NAME = "lora_train_data.csv"
FEEDBACK_FILE_NAME = "feedback.csv"

RANDOM_STATE = 42

DATASET_SAVEPATH = "/content/lora_train_data"
HF_DATASET_REPO = "il-pugin/hse-porg-task-transformer-lora-dataset"

In [4]:
import os
import pandas as pd


train_dataset_filepath = os.path.join(DATASET_DIR_PATH, FILE_NAME)
train_dataset_df = pd.read_csv(train_dataset_filepath)
train_dataset_df.head()

,messages,prompt_id,prompt
0,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",25,\nТы помощник по реформулировке учебных задани...
1,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",26,\nТы помощник по реформулировке учебных задани...
2,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",4,\nТы помощник по реформулировке учебных задани...
3,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",5,\nТы помощник по реформулировке учебных задани...
4,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",6,\nТы помощник по реформулировке учебных задани...


In [5]:
import pandas as pd
import json


def format_df(df: pd.DataFrame) -> pd.DataFrame:
  output_df = df[['prompt_id', 'messages']].copy()
  output_df['dataset'] = output_df['prompt_source'] = 'hse-prog-task-transformer'
  output_df["messages"] = output_df["messages"].apply(json.loads)
  return output_df

In [6]:
formatted_train_dataset_df = format_df(train_dataset_df)


formatted_train_dataset_df.head()

,prompt_id,messages,dataset,prompt_source
0,25,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
1,26,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
2,4,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
3,5,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
4,6,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer


In [9]:
import os
import pandas as pd


test_dataset_filepath = os.path.join(DATASET_DIR_PATH, FEEDBACK_FILE_NAME)
test_dataset_df = pd.read_csv(test_dataset_filepath)
test_dataset_df.head()

,base_model_name,model_version,user_group_cd,messages,prompt_id,prompt,response_id,response,helpful,does_not_reveal_answer,does_not_contain_errors,only_relevant_info
0,deepseek-ai/DeepSeek-R1,0,art-med-com,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",1,\nТы помощник по реформулировке учебных задани...,2,"**Условие задачи** \nПредставьте, что Оля выб...",3,4,True,1
1,deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct,0,art-med-com,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",1,\nТы помощник по реформулировке учебных задани...,30,"<span class="""" style=""font-family: &quot;Trebu...",4,4,True,2
2,deepseek-ai/DeepSeek-R1,0,art-med-com,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",1,\nТы помощник по реформулировке учебных задани...,98,"Давайте представим, что Оля выбирает шоколадку...",1,5,False,1
3,deepseek-ai/DeepSeek-R1,0,soc-econ-sci,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",10,\nТы помощник по реформулировке учебных задани...,11,**Задание: Помощь в выборе домашнего питомца**...,2,3,False,1
4,deepseek-ai/DeepSeek-R1,0,soc-econ-sci,"[{""role"": ""system"", ""content"": ""\u0422\u044b \...",10,\nТы помощник по реформулировке учебных задани...,11,**Задание: Помощь в выборе домашнего питомца**...,5,4,True,4


In [13]:
test_dataset_df = test_dataset_df.drop_duplicates(subset='prompt_id')

In [17]:
test_dataset_df = format_df(test_dataset_df)

In [24]:
test_dataset_df.head()

,prompt_id,messages,dataset,prompt_source
0,1,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
3,10,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
304,4,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
577,7,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer
578,5,"[{'role': 'system', 'content': 'Ты опытный пре...",hse-prog-task-transformer,hse-prog-task-transformer


In [19]:
from datasets import Dataset, DatasetDict


dataset_train = Dataset.from_pandas(formatted_train_dataset_df)
dataset_test = Dataset.from_pandas(test_dataset_df)

#  Create DatasetDict with standard splits
dataset = DatasetDict({
    "train": dataset_train,
    "test": dataset_test
})

# Save as Parquet files
dataset.save_to_disk(DATASET_SAVEPATH)

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12 [00:00<?, ? examples/s]

In [20]:
import os
import fnmatch


# These files mess up dataset structure recognition
def delete_state_json_files(dir_path, state_filename: str = 'state.json'):
    for root, dirs, files in os.walk(dir_path):
        for filename in fnmatch.filter(files, state_filename):
            file_path = os.path.join(root, filename)
            os.remove(file_path)
            print(f'Deleted: {file_path}')

In [21]:
delete_state_json_files(DATASET_SAVEPATH)

Deleted: /content/lora_train_data/test/state.json
Deleted: /content/lora_train_data/train/state.json


In [23]:
# Upload to Hugging Face Hub
from huggingface_hub import HfApi
from google.colab import userdata


api = HfApi(token=userdata.get("HF_TOKEN"))

# Upload dataset
api.upload_folder(
    folder_path=DATASET_SAVEPATH,
    repo_id=HF_DATASET_REPO,
    repo_type="dataset",
)

data-00000-of-00001.arrow:   0%|          | 0.00/75.6k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

data-00000-of-00001.arrow:   0%|          | 0.00/306k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/il-pugin/hse-porg-task-transformer-lora-dataset/commit/d06d7f6eb0967bd0b2d94f9ce8a76c8a2cd1e30a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d06d7f6eb0967bd0b2d94f9ce8a76c8a2cd1e30a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/il-pugin/hse-porg-task-transformer-lora-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='il-pugin/hse-porg-task-transformer-lora-dataset'), pr_revision=None, pr_num=None)